## 10. Advanced Topics
### 10.1 Circular References
#### Context

Description
* in some languages, such as C# or Java, circular references are easy
* but in Rust, they are complicated because of the borrow checker & object lifetime(s) 
* a circular reference occures when two functions call on each other
  * if there is a bad implementation (i.e. no termination condition), then the code could run in a infinite loop

Example
* imagine a class-scheduling programm: where each student can take multiple courses and a course can have multiple students
  * a user will want to 1) call a student to see what courses they are enrolled in and 2) call a course to see which students are attending it
  * there relationship is n:n
  * `struct Student` and `struct Course` will need to have the same lifetime 

<p align="center"><img src="circular_reference_example.png" ></p>

#### Non-Working Example

```Rust
// circular implementation with generic lifetimes
// this is not going to work in Rust

struct Student<'a>{
    name: String,
    courses: Vec<&'a Course<'a>>
}

impl<'a> Student<'a>{
    fn new(name: &str) -> Student<'a>{
        Student(name: name.into(), courses: Vec::new())}
}

struct Course<'a>{
    name: String,
    students: Vec<&'a Student<'a>>
}

impl<'a> Course<'a>{
    fn new(name: &str) -> Course<'a>{
        Course(name: name.into(), students: Vec::new())}
    fn add_student(&'a mut self, student:&'a mut Student<'a>){
        student.courses.push(self);
        self.students.push(student);
    }
}
```

#### `RefCell` Solution 
* however, circular referencing and borrowing of lifetimes is not recommended (as native references are static)
* using the reference counter `std::rc::Rc` and the reference cell `std:cell::RefCell`, we can implement this
* the [RefCell](https://doc.rust-lang.org/std/cell/index.html#refcellt) provides "dynamic borrowing" 
* immutable references can be called from `RefCell` with `.borrow()` and mutuable ones with `-borrow_mut()`
* but we are losing the static borrow checks during compulation, but only we have some run-time checks

In [2]:
use std::rc::Rc;
use std::cell::RefCell;

struct Student {
    name: String,
    courses: Vec<Rc<RefCell<Course>>>
}

impl Student {
    fn new(name: &str) -> Student{
        Student{name: name.into(), courses: Vec::new(),}
    }
}
 
struct Course {
    name: String,
    students: Vec<Rc<RefCell<Student>>>
}

impl Course {
    fn new(name: &str) -> Course{
        Course{name: name.into(), students: Vec::new(),}
    }

    fn add_student(course: Rc<RefCell<Course>>, student: Rc<RefCell<Student>>){
        student.borrow_mut().courses.push(course.clone());
        course.borrow_mut().students.push(student);
    }
}

In [3]:
fn circular_demo(){
    let john = Rc::new(RefCell::new(Student::new("John")));

    let math = Course::new("Math");
    let magic_course = Rc::new(RefCell::new(math));

    Course::add_student(magic_course, john);
}

circular_demo()

()

#### Normalization Solution
* when working with databases, n:n relations and circular references are avoided by creating an intermediary table that connects the pair of previous entities
* this is called normalization
* for our example, this means
  * the `struct Student` only holds a student's name
  * the `struct Course` only holds a course's name
  * the `struct Enrollment` holds a pair of student and course
  * the `struct Platform` that adds and tracks the enrollments 
* here, we need to pay attention again to lifetimes though

In [2]:
struct Student{name: String}
struct Course{name: String}

struct Enrollment<'a>{
    // enrollment pairing
    student: &'a Student,
    course: &'a Course
}

impl<'a> Enrollment<'a> {
    fn new(student: &'a Student, course: &'a Course) -> Enrollment<'a>
    {
        Enrollment{student, course}
    }
}

struct Platform<'a> {
    // platform to track enrollments
    enrollments: Vec<Enrollment<'a>>
}

impl<'a> Platform<'a> {
    // create new platform
    fn new() -> Platform<'a>{
        Platform {enrollments: Vec::new()}
    }
    // adding a enrollment
    fn enroll(&mut self, student: &'a Student, course: &'a Course){
        self.enrollments.push(Enrollment::new(student, course))
    }
}

impl Student {
    fn get_courses(&self, platform: Platform) -> Vec<String>{
        platform.enrollments.iter()
            // from all enrollments find this student
            .filter(|&e| e.student.name == self.name)
            // copy all of their courses
            .map(|e| e.course.name.clone())
            // put the courses into vector
            .collect() 
    }
}

In [3]:
fn normalization_demo(){
    let john = Student{name: "John".into()};
    let math = Course{name: "Math".into()};
    let eng = Course{name: "English".into()};

    let mut plat = Platform::new();
    plat.enroll(&john, &math); 
    plat.enroll(&john, &eng);

    for c in john.get_courses(plat){
        println!("{} is taking {} classes.", john.name, c);
    }
}

normalization_demo()

John is taking Math classes.
John is taking English classes.


()